0. Imports

In [15]:
import pandas as pd
import numpy as np

1. Load CSV

In [16]:
# change to your file location
df = pd.read_csv('/content/drive/MyDrive/Škola/DM/evalvacia_modelu_I_a/MLM_vstup.csv', ';', usecols=range(0,13))
df_stats = pd.read_csv('/content/drive/MyDrive/Škola/DM/evalvacia_modelu_I_a/MLM_STUD_stats.csv', ';', usecols=range(0,10))

# fiter for students
df = df[(df['HODINA'] > 6) & (df['HODINA'] <= 22) & (df['STUD'] == 1) & (df['KATEGORIA'].isin(['uvod', 'studium', 'o_fakulte', 'oznamy']))]

# empty dict to save created crosstables
dfDict = {}

2. Create crosstables

*Crosstable - PO*

In [27]:
df1 = df[(df['PO'] == 1)]
crosstable = pd.crosstab(df1['HODINA'], df1['KATEGORIA'], values=df1['PO'], margins=True, 
                 dropna=False, aggfunc='count').reset_index().fillna(0)

# Add missing line
crosstable = crosstable.append({'HODINA': 7, 'o_fakulte': 0, 'oznamy': 0, 'studium': 0, 'uvod': 0, 'All': 0}, ignore_index=True)

# Add PO crosstable into dict
dfDict['PO'] = crosstable

*Crosstable - UT*

In [18]:
df1 = df[(df['UT'] == 1)]
crosstable = pd.crosstab(df1['HODINA'], df1['KATEGORIA'], values=df1['UT'], margins=True, 
                 dropna=False, aggfunc='count').reset_index().fillna(0)

# Add UT crosstable into dict
dfDict['UT'] = crosstable

*Crosstable - STR*

In [19]:
df1 = df[(df['STR'] == 1)]
crosstable = pd.crosstab(df1['HODINA'], df1['KATEGORIA'], values=df1['STR'], margins=True, 
                 dropna=False, aggfunc='count').reset_index().fillna(0)

# Add missing line   
crosstable = crosstable.append({'HODINA': 22, 'o_fakulte': 0, 'oznamy': 0, 'studium': 0, 'uvod': 0, 'All': 0}, ignore_index=True)

# Add STR crosstable into dict
dfDict['STR'] = crosstable

*Crosstable - STVR*

In [20]:
df1 = df[(df['STVR'] == 1)]
crosstable = pd.crosstab(df1['HODINA'], df1['KATEGORIA'], values=df1['STVR'], margins=True, 
                 dropna=False, aggfunc='count').reset_index().fillna(0)

# Add STVR crosstable into dict
dfDict['STVR'] = crosstable

*Crosstable - PIA*

In [21]:
df1 = df[(df['PIA'] == 1)]
crosstable = pd.crosstab(df1['HODINA'], df1['KATEGORIA'], values=df1['PIA'], margins=True, 
                 dropna=False, aggfunc='count').reset_index().fillna(0)

# Add missing lines
crosstable = crosstable.append({'HODINA': 18, 'o_fakulte': 0, 'oznamy': 0, 'studium': 0, 'uvod': 0, 'All': 0}, ignore_index=True)
crosstable = crosstable.append({'HODINA': 21, 'o_fakulte': 0, 'oznamy': 0, 'studium': 0, 'uvod': 0, 'All': 0}, ignore_index=True)

# Add PIA crosstable into dict
dfDict['PIA'] = crosstable

*Crosstable - SO*

In [22]:
df1 = df[(df['SO'] == 1)]
crosstable = pd.crosstab(df1['HODINA'], df1['KATEGORIA'], values=df1['SO'], margins=True, 
                 dropna=False, aggfunc='count').reset_index().fillna(0)

# Add missing lines
crosstable = crosstable.append({'HODINA': 17, 'o_fakulte': 0, 'oznamy': 0, 'studium': 0, 'uvod': 0, 'All': 0}, ignore_index=True)
crosstable = crosstable.append({'HODINA': 18, 'o_fakulte': 0, 'oznamy': 0, 'studium': 0, 'uvod': 0, 'All': 0}, ignore_index=True)
crosstable = crosstable.append({'HODINA': 19, 'o_fakulte': 0, 'oznamy': 0, 'studium': 0, 'uvod': 0, 'All': 0}, ignore_index=True)
crosstable = crosstable.append({'HODINA': 20, 'o_fakulte': 0, 'oznamy': 0, 'studium': 0, 'uvod': 0, 'All': 0}, ignore_index=True)

# Add SO crosstable into dict
dfDict['SO'] = crosstable

3. Create collection of weekdays

In [23]:
days = ['PO', 'UT', 'STR', 'STVR', 'PIA', 'SO']

4. Create estimates for web parts

In [24]:
df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
df4 = pd.DataFrame()
index = 0

# Cycle through hours from 7 to 23
for x in range (7,23):
      new_row_uvod = {}
      new_row_studium = {}
      new_row_oznamy = {}
      new_row_fakulte = {}
      i = 1

      # Cycle through weekdays
      for day in days:

        # Create logits estimates
        logit_uvod = df_stats.at[index, 'Intercept'] + df_stats.at[index, 'HODINA']*x+df_stats.at[index, 'HODINA_STV']*(x*x)+df_stats.at[index, day]
        logit_studium = df_stats.at[index+1, 'Intercept'] + df_stats.at[index+1, 'HODINA']*x+df_stats.at[index+1, 'HODINA_STV']*(x*x)+df_stats.at[index+1, day]
        logit_oznamy = df_stats.at[index+2, 'Intercept'] + df_stats.at[index+2, 'HODINA']*x+df_stats.at[index+2, 'HODINA_STV']*(x*x)+df_stats.at[index+2, day]
        
        reference_web = 1 / (1 + np.exp(logit_uvod) + np.exp(logit_studium) + np.exp(logit_oznamy))
        
        # Create estimates for web parts
        estimate_uvod = np.exp(logit_uvod) * reference_web
        estimate_studium = np.exp(logit_studium) * reference_web
        estimate_oznamy = np.exp(logit_oznamy) * reference_web
        estimate_fakulte = np.exp(reference_web) * reference_web

        # Create evaluation
        crosstable = dfDict[day]
        crosstable = crosstable[(crosstable['HODINA'] == x)]
        evaluation_uvod = estimate_uvod * crosstable.iloc[0]['All']
        evaluation_studium = estimate_studium * crosstable.iloc[0]['All']
        evaluation_oznamy = estimate_oznamy * crosstable.iloc[0]['All']
        evaluation_fakulte = estimate_fakulte * crosstable.iloc[0]['All']

        den = str(i) + '_' + day

        # Create new rows and append it to dataframe
        new_row_uvod.update({den: evaluation_uvod})
        new_row_studium.update({den: evaluation_studium})
        new_row_oznamy.update({den: evaluation_oznamy})
        new_row_fakulte.update({den: evaluation_fakulte})
        i = i + 1
      
      # Append time to rows
      new_row_uvod.update({'0_hod': x})
      new_row_studium.update({'0_hod': x})
      new_row_oznamy.update({'0_hod': x})
      new_row_fakulte.update({'0_hod': x})

      # Update dataframes
      df1 = df1.append(new_row_uvod, sort=False, ignore_index=True)
      df2 = df2.append(new_row_studium, sort=False, ignore_index=True)
      df3 = df3.append(new_row_oznamy, sort=False, ignore_index=True)
      df4 = df4.append(new_row_fakulte, sort=False, ignore_index=True)

df1.head(16)

,0_hod,1_PO,2_UT,3_STR,4_STVR,5_PIA,6_SO
0,7.0,0.000000,3.571617,26.451747,11.525116,8.553339,10.399792
1,8.0,13.833549,13.068825,18.653568,3.264149,89.731902,76.807945
2,9.0,64.192605,54.199823,36.392573,38.440714,83.354462,28.419364
3,10.0,67.507120,63.646036,9.563301,17.905450,30.561764,33.015756
4,11.0,66.064018,73.589841,58.256822,14.234026,43.808187,68.355672
5,12.0,75.434503,56.879803,38.921935,5.701469,30.447926,21.304927
6,13.0,55.793916,36.775645,38.453316,42.139338,64.659762,28.288729
7,14.0,75.843812,43.592840,50.723343,41.390634,22.851268,8.942736
8,15.0,96.626297,6.921608,14.748775,6.718350,10.555898,21.760039
9,16.0,11.959967,17.299181,48.760589,23.328197,3.077947,31.903484


4. Export to excel

In [25]:
# Creating Excel Writer Object from Pandas  
writer = pd.ExcelWriter('STUD_evaluation_model.xlsx',engine='xlsxwriter')   
workbook=writer.book
worksheet=workbook.add_worksheet('STUD')
writer.sheets['STUD'] = worksheet

# Úvod
worksheet.write(0, 0, "Úvod")
df1.to_excel(writer, sheet_name='STUD',startrow=1 , startcol=0, index=False)

# Śtúdium
worksheet.write(0, 8, "Štúdium")
df2.to_excel(writer, sheet_name='STUD',startrow=1 , startcol=8, index=False)

# Oznamy
worksheet.write(0, 16, "Oznamy")
df3.to_excel(writer, sheet_name='STUD',startrow=1 , startcol=16, index=False)

# O fakulte
worksheet.write(0, 24, "O fakulte")
df4.to_excel(writer, sheet_name='STUD',startrow=1 , startcol=24, index=False)
writer.save()